In [22]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# import
import pandas as pd
from pprint import pprint
from zen_garden.postprocess.results import Results
from pathlib import Path
from os import chdir

# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
# define import path
path_import = Path("../data/outputs/04_CCTS_WTE_Cement_01_optimal_design")

# define export path
path_export = Path("/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/04_CCTS_WTE_Cement_02_failure")

In [24]:
# cost optimal solution
results = Results(path_import)

In [25]:
# get flow transport and save as csv
results.get_df('flow_transport', to_csv="flow_transport")

technology   edge                         time_operation
flowline     waste_5_CH-swiss_storage_CH  0                 0.0
                                          1                 0.0
                                          2                 0.0
                                          3                 0.0
                                          4                 0.0
                                                           ... 
truck_batch  waste_9_CH-waste_8_CH        5                 0.0
                                          6                 0.0
                                          7                 0.0
                                          8                 0.0
                                          9                 0.0
Name: flow_transport, Length: 21060, dtype: float64

In [26]:
# get flow transport csv
df = pd.read_csv('flow_transport.csv')

In [27]:
# create nominal flow transport files

# find unique technologies
technology = df['technology'].unique()

for i in range(len(technology)):
     
    #Get table of one technology 
    tec = technology[i]
    x = df[df['technology'] == tec]
    
    #Set path
    parent_path = Path(path_export)
    path = [file for file in Path(parent_path).rglob(tec)]
    path = path[0]
    filepath = Path(str(path)+'/nominal_flow_transport.csv')
    
    #Clean up table
    x = x.drop(columns='technology')
    x = x[x['flow_transport'] != 0]
    x = x.rename(columns={'time_operation': 'year'})
    x['year'] = x['year'] + 2025 
    
    #clean up table, differentiate between node and edge
    if path.parent.name == 'set_transport_technologies':
        x = x.rename(columns={'location': 'edge'})

    else:
        x = x.rename(columns={'location': 'node'})

    #Safe
    x.to_csv(filepath, index=False)